In [ ]:
import bagpipes as pipes
import numpy as np
import pymultinest

In [ ]:
filter_list = ["filters/nircam_throughputs/mean_throughputs/F090.dat", "filters/nircam_throughputs/mean_throughputs/F115.dat","filters/nircam_throughputs/mean_throughputs/F150.dat","filters/nircam_throughputs/mean_throughputs/F200.dat", "filters/nircam_throughputs/mean_throughputs/F277.dat","filters/nircam_throughputs/mean_throughputs/F356.dat", "filters/nircam_throughputs/mean_throughputs/F410.dat", "filters/nircam_throughputs/mean_throughputs/F444.dat","filters/HST/wfc3-F125W.res","filters/HST/wfc3-F140W.res","filters/HST/wfc3-F160W.res","filters/HST/ACS_F435W.res", "filters/HST/ACS_F606W.res", "filters/HST/ACS_F814W.res"]

In [ ]:
def load_data(ID):
    # load up the relevant columns from the catalogue.
    cat = np.loadtxt("jwst_hst_photometry.txt")
    cat_err = np.loadtxt("jwst_hst_photometry_err.txt")
    # Find the correct row for the object we want.
    row = int(ID) 
    # Extract the object we want from the catalogue.
    fluxes = cat[row]/1000.
    fluxerrs = cat_err[row]/1000.
#    print(fluxes) 
#    print(fluxerrs) 
    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]
    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0):
            photometry[i,:] = [0., 9.9*10**20.]
    print(photometry)
    return photometry

In [ ]:
exp = {}                                  # Tau-model star-formation history component
exp["age"] = (0.01, 15.)                   # Vary age between 100 Myr and 15 Gyr. In practice
                                             # the code automatically limits this to the age of
                                             # the Universe at the observed redshift.

In [ ]:
exp["tau"] = (0.01, 5.)                   # Vary tau between 300 Myr and 10 Gyr
exp["massformed"] = (0.1, 12.)             # vary log_10(M*/M_solar) between 1 and 15
exp["metallicity"] = (0., 2.5)            # vary Z between 0 and 2.5 Z_oldsolar

In [ ]:
dust = {}                                 # Dust component
dust["type"] = "Calzetti"                 # Define the shape of the attenuation curve
dust["Av"] = (0.0, 2.)                     # Vary Av between 0 and 2 magnitudes

In [ ]:
nebular = {}                      # Nebular emission component
nebular["logU"] = -2.5             # log_10(ionization parameter)

In [ ]:
fit_instructions = {}                     # The fit instructions dictionary
fit_instructions["redshift"] = (0., 3.)  # Vary observed redshift from 0 to 10
fit_instructions["exponential"] = exp
fit_instructions["dust"] = dust
fit_instructions["nebular"] = nebular

In [ ]:
IDs = np.arange(0,5)

In [ ]:
fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_data, spectrum_exists=False, make_plots=True,
                              cat_filt_list=filter_list, run="all_cat1")
fit_cat.fit(verbose=False)